# Process SOM-FFN floats

In [1]:
import xarray as xr 
import stats_tools
import decompose as stl
import matplotlib.pyplot as plt
import cmocean as cm
from plotting_tools.spatial_map import SpatialMap

import cmocean as cm
from plotting_tools.spatial_map import SpatialMap
import plotting_tools
import numpy as np

In [2]:
from skill_metrics import skill_metrics as sk

In [3]:
def decompose_the_signal(data=None, lo_pts=None, lo_delta=None):
    ###======================================
    ### STL Decomposition
    ###======================================
    ### 0. Load raw data
    data = da.copy()

    #print('detrend')
    ### 1. Detrend it 
    data_detrend = stl.detrend(data, dim='time')

    #print('deseason')
    ### 2. Remove seasonal cycle
    # 2.1 -- seasonal cycle
    data_seasonal = stl.seasonal_cycle(data_detrend, dim='time', period=12)

    #print('deseason')
    # 2.2 -- de-season the data
    data_deseason = data_detrend - data_seasonal

    #print('lowess')
    ### 3. calculate the LOWESS
    data_lowess = stl.lowess(data_deseason, dim='time', lo_pts=lo_pts, lo_delta=lo_delta)

    #print('residual')
    ### 4. Residual term -- not explained by trend or seasonal cycle 
    data_residual = data_deseason - data_lowess

    ### Low frequency of residual
    #data_residual_low = stl.lowess(data_residual, dim='time', lo_pts=12, lo_delta=0.01)

    # Save data as dataset
    ###======================================
    ### Return dataset
    ###======================================
    ds_out = xr.Dataset(
        {    
        f'{var_name}': (['time', 'lat', 'lon'], data ),
        f'{var_name}_detrend': (['time', 'lat', 'lon'], data_detrend ),
        f'{var_name}_decadal': (['time','lat', 'lon'], data_lowess ),
        f'{var_name}_seasonal': (['time','lat', 'lon'], data_seasonal ),
        f'{var_name}_residual': (['time','lat', 'lon'], data_residual ),
        #f'{var_out}_residual_low': (['time','lat', 'lon'], data_residual_low ),
        },

        coords={
        'time': (['time'], da['time']),
        'lat': (['lat'], da['lat']),
        'lon': (['lon'], da['lon']),
        })
    
    return ds_out

    
# decompose or dask
def decompose_dask(model=None, 
                  mem=None, 
                  suffix=None,
                  var_name=None, 
                  dir_out=None):
    '''
    decompose_stl(da=None, var_name=None)
        decompose a DataArray using STL method.
        
    Input
    ==========
    da : datarray
    var_name : name of variable for output
    mem : member number (eg. '001')
    model : model name (eg. 'CESM')
    suffix : ending suffix (eg. 'SOMFFM-float')
    dir_out : output directory (eg. './output')
    Output 
    ==========
    ds : dataset with original signal and decomposed
    
    Reference
    ==========
    Cleveland et al. 1990
    '''
    # define stl parameters
    lo_pts=12*10
    lo_delta=0.01
            
    ###======================================
    ### Load data
    ###======================================
    # define path
    dir_data = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/raw'
    dir_float = f'{dir_data}/interpolated/SOMFFN_float'
    # load data
    ds = xr.open_dataset(f'{dir_float}/{model}/pCO2_2D_mon_{model}{mem}_1x1_198201-201701_SOMFFN_float.nc')
    da = ds['pCO2_SOMFFN']
    
    # roll longitude
    da = da.roll(lon=180, roll_coords='lon')
    da['lon'] = np.arange(0.5,360,1)

    ###======================================
    ### decompose signal
    ###======================================
    ds_out = decompose_the_signal(data=da, lo_pts=lo_pts, lo_delta=lo_delta)
    
    #return ds_out
    ds_out.to_netcdf(f'{dir_out}/{var_name}_decomp_{model}{mem}_{suffix}.nc')
    #ds_out.to_netcdf(f'{dir_out}/pco2_decomp_{model}{mem}_{model_or_recon}.nc') 

In [4]:
def decompose_stl(model=None, 
                  mem=None, 
                  suffix=None,
                  var_name=None, 
                  dir_out=None):
    '''
    decompose_stl(da=None, var_name=None)
        decompose a DataArray using STL method.
        
    Input
    ==========
    da : datarray
    var_name : name of variable for output
    mem : member number (eg. '001')
    model : model name (eg. 'CESM')
    suffix : ending suffix (eg. 'SOMFFM-float')
    dir_out : output directory (eg. './output')
    Output 
    ==========
    ds : dataset with original signal and decomposed
    
    Reference
    ==========
    Cleveland et al. 1990
    '''
    # define stl parameters
    lo_pts=12*10
    lo_delta=0.01
            
    ###======================================
    ### Load data
    ###======================================
    # define path
    dir_data = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/raw'
    dir_float = f'{dir_data}/interpolated/SOMFFN_float'
    # load data
    ds = xr.open_dataset(f'{dir_float}/{model}/pCO2_2D_mon_{model}{mem}_1x1_198201-201701_SOMFFN_float.nc')
    da = ds['pCO2_SOMFFN']
    
    # roll longitude
    da = da.roll(lon=180, roll_coords='lon')
    da['lon'] = np.arange(0.5,360,1)

    ###======================================
    ### STL Decomposition
    ###======================================
    ### 0. Load raw data
    data = da.copy()

    #print('detrend')
    ### 1. Detrend it 
    data_detrend = stl.detrend(data, dim='time')

    #print('deseason')
    ### 2. Remove seasonal cycle
    # 2.1 -- seasonal cycle
    data_seasonal = stl.seasonal_cycle(data_detrend, dim='time', period=12)

    #print('deseason')
    # 2.2 -- de-season the data
    data_deseason = data_detrend - data_seasonal

    #print('lowess')
    ### 3. calculate the LOWESS
    data_lowess = stl.lowess(data_deseason, dim='time', lo_pts=lo_pts, lo_delta=lo_delta)

    #print('residual')
    ### 4. Residual term -- not explained by trend or seasonal cycle 
    data_residual = data_deseason - data_lowess

    ### Low frequency of residual
    #data_residual_low = stl.lowess(data_residual, dim='time', lo_pts=12, lo_delta=0.01)

    # Save data as dataset
    ###======================================
    ### Return dataset
    ###======================================
    ds_out = xr.Dataset(
        {    
        f'{var_name}': (['time', 'lat', 'lon'], data ),
        f'{var_name}_detrend': (['time', 'lat', 'lon'], data_detrend ),
        f'{var_name}_decadal': (['time','lat', 'lon'], data_lowess ),
        f'{var_name}_seasonal': (['time','lat', 'lon'], data_seasonal ),
        f'{var_name}_residual': (['time','lat', 'lon'], data_residual ),
        #f'{var_out}_residual_low': (['time','lat', 'lon'], data_residual_low ),
        },

        coords={
        'time': (['time'], da['time']),
        'lat': (['lat'], da['lat']),
        'lon': (['lon'], da['lon']),
        })
    
    #return ds_out
    ds_out.to_netcdf(f'{dir_out}/{var_name}_decomp_{model}{mem}_{suffix}.nc')
    #ds_out.to_netcdf(f'{dir_out}/pco2_decomp_{model}{mem}_{model_or_recon}.nc') 

In [5]:
# Dictionary of float members
# SOME ARE NOT IN THE 25 MEMBERS
# ASK PETER FOR 3 from CanESM2 and 3 from MPI total of 20 members 
#float_files = {'CESM':['001','010','011','014','017','035','103'],
#              'CanESM2':['r4r3','r5r5'], #'r1r4','r1r8','r2r3', 'r3r8','r4r10',
#              'GFDL':['02','03','08','19','27'], # '15', '24'
#              'MPI':['027', '064']} # '018','021', '028', '039', '041',


In [6]:
# members that were SOCAT+float sampeled
float_files = {              
    'CanESM2':['r1r4','r1r8','r2r3', 'r3r8','r4r10','r4r3','r5r5'],
    'GFDL':['15', '24'], 
    'MPI':['018','021', '028', '039', '041', '027', '064'],
                } 

## Decompose pCO2

In [7]:
from distributed import Client

In [8]:
# 1. Intitialize the dask client
client = Client(n_workers=20)
# 2. Start the dask client. 
client

Client Scheduler: tcp://127.0.0.1:44969 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 20 Cores: 60 Memory: 269.92 GB


In [8]:
###======================================
### model_or_recon needs to be either MODEL or SOMFFN
###====================================== 
model = 'MPI'
members = float_files[model]
n = len(members)
suffix = 'SOMFFN-float'
var_name='spco2'
dir_out = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/pCO2-float_decomp_stl'

## Run the decompose_pco2 function
fut = client.map(decompose_stl, 
                 tuple(np.repeat(model, n)), 
                 members, 
                 tuple(np.repeat(suffix, n)),
                 tuple(np.repeat(var_name, n)),
                 tuple(np.repeat(dir_out, n)))
client.gather(fut)

del fut
    
print('done!')

done!


# Calculate pCO2 stats

In [10]:
def calculate_pco2_statistics(model=None, member=None, dir_out=None):
    """
    calculate_pco2_statistics(model=None, member=None)
    model   = [MPI, CESM, or GFDL]
    member  = "member you want to process"
    dir_in  = directory for CO2 inputs (do not put / at end)
    dir_out = directory for output (do not put / at end) 
    
    Notes
    =========
    
    
    L. Gloege 2019
    """
    ### ================================================
    ### Get file path
    ### ================================================
    #if model.upper()=='CESM':
    #    mem = f'{member:0>3}'
    #if model.upper()=='GFDL':
    #    mem = f'{member:0>2}'
    #if model.upper()=='MPI':
    #    mem = f'{member:0>3}'
    #if model.upper()=='CANESM2':
    #    mem = f'{member}'
    
    ###======================================
    ### Load model
    ###====================================== 
    
    dir_float='/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/pCO2-float_decomp_stl'
    dir_mod='/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/pCO2_decomp_stl'
    
    ds_som = xr.open_dataset(f'{dir_float}/spco2_decomp_{model}{member}_SOMFFN-float.nc')  
    
    ds_mod = xr.open_dataset(f'{dir_mod}/pco2_decomp_{model}{member}_MODEL.nc')
        
    ###======================================
    ### Return dataset
    ###======================================
    ds_out = xr.Dataset(
        {
        ## Bias
        'bias': (['lat', 'lon'], sk.avg_error(ds_mod['pco2'], 
                                              ds_som['spco2']),
                 {'units':'uatm'}), 
        'bias_detrend':(['lat','lon'], sk.avg_error(ds_mod['pco2_detrend'], 
                                                    ds_som['spco2_detrend']),
                       {'units':'uatm'}),
        'bias_dec': (['lat', 'lon'], sk.avg_error(ds_mod['pco2_dec'], 
                                                    ds_som['spco2_decadal']),
                       {'units':'uatm'}),
        'bias_seasonal': (['lat', 'lon'], sk.avg_error(ds_mod['pco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
                       {'units':'uatm'}),
        'bias_residual': (['lat', 'lon'], sk.avg_error(ds_mod['pco2_residual'], 
                                                      ds_som['spco2_residual']),
                       {'units':'uatm'}),
        #'bias_residual_low': (['lat', 'lon'], sk.avg_error(ds_mod['pco2_residual_low'], 
        #                                              ds_som['spco2_residual_low']),
        #               {'units':'uatm'}),

        ## average absolute error
        'aae': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2'], 
                                              ds_som['spco2']),
                 {'units':'uatm'}), 
        'aae_detrend':(['lat','lon'], sk.avg_abs_error(ds_mod['pco2_detrend'], 
                                                    ds_som['spco2_detrend']),
                       {'units':'uatm'}),
        'aae_dec': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2_dec'], 
                                                    ds_som['spco2_decadal']),
                       {'units':'uatm'}),
        'aae_seasonal': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
                       {'units':'uatm'}),
        'aae_residual': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2_residual'], 
                                                      ds_som['spco2_residual']),
                       {'units':'uatm'}),
        #'aae_residual_low': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2_residual_low'], 
        #                                              ds_som['pco2_residual_low']),
        #               {'units':'uatm'}),
            
       
        ## Amplitude ratio
        'amp-ratio': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2'], 
                                              ds_som['spco2']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_detrend':(['lat','lon'], sk.amp_ratio(ds_mod['pco2_detrend'], 
                                                    ds_som['spco2_detrend']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_dec': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2_dec'], 
                                                    ds_som['spco2_decadal']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_seasonal': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_residual': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2_residual'], 
                                                      ds_som['spco2_residual']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        #'amp-ratio_residual_low': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2_residual_low'], 
        #                                              ds_som['pco2_residual_low']),
        #      {'units':'dimensionless',
        #       'notes':'[(amp_rec/amp_mod) - 1]'}),
            
        
        ## correlation
        'corr': (['lat', 'lon'], sk.correlation(ds_mod['pco2'], 
                                              ds_som['spco2']),
            {'units':'dimensionless'}),
        'corr_detrend':(['lat','lon'], sk.correlation(ds_mod['pco2_detrend'], 
                                                    ds_som['spco2_detrend']),
            {'units':'dimensionless'}),
        'corr_dec': (['lat', 'lon'], sk.correlation(ds_mod['pco2_dec'], 
                                                    ds_som['spco2_decadal']),
            {'units':'dimensionless'}),
        'corr_seasonal': (['lat', 'lon'], sk.correlation(ds_mod['pco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
            {'units':'dimensionless'}),
        'corr_residual': (['lat', 'lon'], sk.correlation(ds_mod['pco2_residual'], 
                                                      ds_som['spco2_residual']),
            {'units':'dimensionless'}),
      #  'corr_residual_low': (['lat', 'lon'], sk.correlation(ds_mod['pco2_residual_low'], 
      #                                                ds_som['pco2_residual_low']),
      #      {'units':'dimensionless'}),
            
            
        ## Normalized standard deviation
        'std-star': (['lat', 'lon'], sk.std_star(ds_mod['pco2'], 
                                              ds_som['spco2']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_detrend':(['lat','lon'], sk.std_star(ds_mod['pco2_detrend'], 
                                                    ds_som['spco2_detrend']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_dec': (['lat', 'lon'], sk.std_star(ds_mod['pco2_dec'], 
                                                    ds_som['spco2_decadal']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_seasonal': (['lat', 'lon'], sk.std_star(ds_mod['pco2_seasonal'], 
                                                      ds_som['spco2_seasonal']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_residual': (['lat', 'lon'], sk.std_star(ds_mod['pco2_residual'], 
                                                      ds_som['spco2_residual']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
      #  'std-star_residual_low': (['lat', 'lon'], sk.std_sta"r(ds_mod['pco2_residual_low'], 
      #                                                ds_som['pco2_residual_low']),
      #                {'units':'dimensionless',
      #                 'notes':'[(STD_rec/STD_mod) - 1]'})
        },

        coords={
        'lat': (['lat'], ds_mod['lat']),
        'lon': (['lon'], ds_mod['lon']),
        })
        
    # save file
    ds_out.to_netcdf(f'{dir_out}/stats_spco2_{model}{member}_SOMFFN-float.nc') 

In [12]:
###======================================
### model_or_recon needs to be either MODEL or SOMFFN
###====================================== 
model = 'MPI'
members = float_files[model]
n = len(members)
#suffix = 'SOMFFN-float'
#var_name='spco2'
dir_out = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/pCO2-float_stats'

## Run the decompose_pco2 function
fut = client.map(calculate_pco2_statistics, 
                 tuple(np.repeat(model, n)), 
                 members, 
                 tuple(np.repeat(dir_out, n)))
client.gather(fut)

del fut
    
print('done!')

done!


# calculate flux


In [9]:
# these are notebooks with function definitions
%run _define_model_class.ipynb
%run _define_processing_functions.ipynb

In [10]:
def calculate_flux(model=None, 
                   member=None, 
                   fl_u10_std=None, 
                   dir_out=None, ):
    '''
    calculate_flux(model=None, member=None)
    calculate CO2 flux following Wanninkhof (1992) for each member.
    
    Inputs
    ============
    model  = name of model (CESM, CanESM2, GFDL, MPI)
    member = member number
    fl_u10_std = file path to u10-STD netcdf file
    dir_out = directory to store output (do not put "/" at end)
    
    Output
    ============
    None. This stores the calculated flux as a NetCDF file in the {data_dir} location.
    
    Notes
    ============
    This uses the read_model2() function contained in _define_model_class.ipynb
    This uses the scaled version of Sweeney et al. (2007) used in 
    Landschutzer et al. (2016). Scale factor 1.014 is from person comm. with Peter
    '''
    ###======================================
    ### Define directories
    ###====================================== 
    #dir_raw = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/raw'
    #dir_clean = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean'
    
    ### ================================================
    ### Get file path
    ### ================================================

    ### Load auxillary data
    ds_SST     = read_model2(model=model, member=mem, variable='SST')
    ds_SSS     = read_model2(model=model, member=mem, variable='SSS')
    ds_iceFrac = read_model2(model=model, member=mem, variable='iceFrac')
    ds_pATM    = read_model2(model=model, member=mem, variable='pATM')
    ds_XCO2    = read_model2(model=model, member=mem, variable='XCO2')
    ds_U10     = read_model2(model=model, member=mem, variable='U10')
    ds_pCO2    = read_model2(model=model, member=mem, variable='pCO2')
    ds_pCO2_SOMFFN = read_somffn_float(model=model, member=mem)
    
    ### Wind speed variance 
    #ds_U_std = xr.open_dataset(f'{dir_clean}/ERA_interim/ERAinterim_1x1_u10-std_1982-2016.nc', 
    #                           decode_times=False)
    ds_U_std = xr.open_dataset(f'{fl_u10_std}', decode_times=False)
    ###======================================
    ### Put data into xarray dataset
    ###======================================
    ds = xr.Dataset(
        {
        'SST':(['time','lat','lon'], ds_SST['SST'] ),
        'SSS':(['time','lat','lon'], ds_SSS['SSS']),
        'iceFrac':(['time','lat','lon'], ds_iceFrac['iceFrac'] ),
        'U10':(['time','lat','lon'], ds_U10['U10']),
        'U10_std':(['time','lat','lon'], ds_U_std['u10']),
        'Patm':(['time','lat','lon'], ds_pATM['pATM'] ),
        'xCO2':(['time'], ds_XCO2['XCO2']),
        'pCO2_member':(['time','lat','lon'], ds_pCO2['pCO2']),
        'pCO2_somffn':(['time','lat','lon'], ds_pCO2_SOMFFN['pco2']),
        },

        coords={
        'lat': (['lat'], ds_pCO2_SOMFFN['lat']),
        'lon': (['lon'], ds_pCO2_SOMFFN['lon']),
        'time': (['time'], ds_pCO2_SOMFFN['time'])
        })

    F_somffn = wanninkhof92(T=ds['SST'], 
                 S=ds['SSS'], 
                 P=ds['Patm'],
                 u_mean=ds['U10'], 
                 u_std=ds['U10_std'], 
                 xCO2=ds['xCO2'],
                 pCO2_sw = ds['pCO2_somffn'],
                 iceFrac = ds['iceFrac'],
                 scale_factor=0.27*1.014)

    F_member = wanninkhof92(T=ds['SST'], 
                 S=ds['SSS'], 
                 P=ds['Patm'],
                 u_mean=ds['U10'], 
                 u_std=ds['U10_std'], 
                 xCO2=ds['xCO2'],
                 pCO2_sw = ds['pCO2_member'],
                 iceFrac = ds['iceFrac'],
                 scale_factor=0.27*1.014)

    # Save output in file
    ds_out = xr.Dataset(
        {
        'F_member':(['time','lat','lon'], F_member), 
        'F_somffn':(['time','lat','lon'], F_somffn),       
        },

        coords={
        'lat': (['lat'], ds['lat']),
        'lon': (['lon'], ds['lon']),
        'time': (['time'], ds['time'])
        })
        
    # Save to netcdf
    ds_out.to_netcdf(f'{dir_out}/CO2_flux_{model}{mem}_SOMFFN-float.nc')

In [11]:
# Directory with u10_std
dir_clean = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean'
fl_u10_std = f'{dir_clean}/ERA_interim/ERAinterim_1x1_u10-std_1982-2016.nc'

###======================================
### model_or_recon needs to be either MODEL or SOMFFN
###====================================== 
model = 'GFDL'
members = float_files[model]
n = len(members)
suffix = 'SOMFFN-float'
dir_out = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/CO2_flux-float'

fut = client.map(calculate_flux_float, 
                 tuple(np.repeat(model, n)), 
                 members,
                 tuple(np.repeat(fl_u10_std, n)), 
                 tuple(np.repeat(dir_out, n)))
client.gather(fut)

# Clear up the memory
del fut

print('Complete!!')

Complete!!


# Other

In [23]:
all_members = {'CanESM2': = ['r1r10', 'r1r6', 'r1r7', 'r1r9', 'r2r1', 
                            'r2r2', 'r2r8', 'r3r1', 'r3r2', 'r3r4',
                            'r3r6', 'r3r7', 'r3r9', 'r4r1', 'r4r3', 
                            'r4r5', 'r4r6', 'r4r7', 'r4r8', 'r5r1', 
                            'r5r10', 'r5r2', 'r5r4', 'r5r5', 'r5r9']
    'CESM':[f'{i:03d}' for i in [1,2,9,10,11,12,13,14,15,16,17,18,20,21,23,24,25,30,31,34,35,101,102,103,104]]
    'GFDL': [f'{i:02d}' for i in [1,2,3,4,5,6,8,9,10,11,12,13,14,16,17,18,19,20,22,23,26,27,28,29,30]]
    'MPI': [f'{i:03d}' for i in [6,9,14,20,22,24,25,27,38,43,45,46,57,60,64,70,75,77,78,80,81,83,91,95,98]]}

In [ ]:
    if LE_model=='CESM':
        members=[1,2,9,10,11,12,13,14,15,16,17,18,20,21,23,24,25,30,31,34,35,101,102,103,104]
        n = len(members)
        print(f'Loading {n} members from {LE_model}')

    if LE_model=='GFDL':
        members=[1,2,3,4,5,6,8,9,10,11,12,13,14,16,17,18,19,20,22,23,26,27,28,29,30]
        n = len(members)
        print(f'Loading {n} members from {LE_model}')

    if LE_model=='MPI':
        members=[6,9,14,20,22,24,25,27,38,43,45,46,57,60,64,70,75,77,78,80,81,83,91,95,98]
        n = len(members)
        print(f'Loading {n} members from {LE_model}')

    if LE_model=='CanESM2':
        members=['r1r10', 'r1r6', 'r1r7', 'r1r9', 'r2r1', 'r2r2', 'r2r8', 'r3r1', 'r3r2', 'r3r4',
                'r3r6', 'r3r7', 'r3r9', 'r4r1', 'r4r3', 'r4r5', 'r4r6', 'r4r7', 'r4r8', 'r5r1', 
                'r5r10', 'r5r2', 'r5r4', 'r5r5', 'r5r9']
        n = len(members)
        print(f'Loading {n} members from {LE_model}')

In [ ]:
###======================================
### decompose dataset
###======================================
# define path
dir_data = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/raw'
dir_float = f'{dir_data}/interpolated/SOMFFN_float'
dir_out = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/pCO2-float_decomp_stl'
# load data
ds = xr.open_dataset(f'{dir_float}/CESM/pCO2_2D_mon_CESM010_1x1_198201-201701_SOMFFN_float.nc')

# roll longitude
ds = ds.roll(lon=180, roll_coords='lon')
ds['lon'] = np.arange(0.5,360,1)

                    
# decompose signal

ds_out = stl.decompose_stl(da = ds['pCO2_SOMFFN'], 
                       var_name = 'spco2',
                       mem='010',
                       model='CESM',
                       suffix='SOMFFN-float',
                       dir_out = dir_out)